In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive

In [ ]:
import numpy as np
import argparse
import time
import cv2
import os
import matplotlib.pyplot as plt

#  Load the YOLO weights and configuration files.

wtPath = '/content/drive/MyDrive/yolov3.weights'
cfgPath = '/content/drive/MyDrive/yolov3.cfg'

#load the darknet according to weights and cfg.
net = cv2.dnn.readNet(cfgPath, wtPath)

#define colours for every class label from the dataset.

LABELS= []
with open("/content/drive/MyDrive/coco.names",'r') as f:
  LABELS=f.read().splitlines()

In [ ]:
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")

# read the input image
img=cv2.imread("/content/drive/MyDrive/yolov3_test/fff.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
(H, W) = img.shape[:2]

In [ ]:
blob = cv2.dnn.blobFromImage(img, 1 / 255, (320, 320), [0,0,0], 1, crop=False)

In [ ]:
# pass blob of the image from network
net.setInput(blob)

In [ ]:
layerNames = net.getUnconnectedOutLayersNames()

# get the output
Outputs = net.forward(layerNames)


In [ ]:
boundingBoxes = []
confidences = []
classIDs = []

count=0;
# get the bounding boxes and classes from the output
for output in Outputs:
	for detection in output:
		scores = detection[5:]
		classID = np.argmax(scores)
		confidence = scores[classID]
		count+=1;
		if confidence >= 0.0:
			box = detection[0:4] * np.array([W, H, W, H])
			(cX, cY, width, height) = box.astype("int")
	 
			x = int(cX - (width / 2))
			y = int(cY - (height / 2))

			boundingBoxes.append([x, y, int(width), int(height)])
			confidences.append(float(confidence))
			classIDs.append(classID)
# Apply non-max suppression
idxs = cv2.dnn.NMSBoxes(boundingBoxes, confidences, 0.3, 0.1)


# Draw the bounding boxes

if len(idxs) > 0:
	for i in idxs.flatten():
		(w, h) = (boundingBoxes[i][2], boundingBoxes[i][3])
		(x, y) = (boundingBoxes[i][0], boundingBoxes[i][1])
		clr = [int(c) for c in COLORS[classIDs[i]]]
		cv2.rectangle(img, (x, y), (x + w, y + h), clr, 3)
		text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
		cv2.putText(img, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5, clr, 2)

plt.imshow(img)

